In [16]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score

mort = pd.read_csv('../data/data_active.csv', header=0)
mort.head()

,Bo_Age,Ln_Orig,Orig_LTV_Ratio_Pct,Credit_score,First_home,Tot_mthly_debt_exp,Tot_mthly_incm,orig_apprd_val_amt,pur_prc_amt,DTI Ratio,Status,OUTCOME,State,Median_state_inc,UPB>Appraisal
0,37,75650.0,85,669,N,1707,6000,169000,160000,0.284500,Active,non-default,FL,"40,171",0
1,46,390775.0,102,684,N,0,5025,310000,309000,0.000000,Pay-off,non-default,NY,"44,228",1
2,30,112500.0,90,662,Y,1812,4800,177000,176450,0.377500,Pay-off,non-default,CA,"49,894",0
3,24,85250.0,97,647,N,3395,6934,113000,110000,0.489616,Pay-off,non-default,GA,"43,217",0
4,35,114000.0,100,791,N,3801,5504,103000,103000,0.690589,Pay-off,non-default,NH,"57,352",1


## 전처리 과정

In [17]:
## First_home 문자형 데이터 변환
mort['First_home'] = np.where(mort['First_home']=='Y',1,0)

## OUTCOME 데이터 변환
mort['OUTCOME'] = np.where(mort['OUTCOME']=='non-default',0,1) # non-default = 0, default = 1

## string 타입을 정수로 변환하기
for k in range(0,10607):
    mort.Median_state_inc[k] = np.float64(mort.Median_state_inc[k].replace(",", ""))
    
outcome = mort.OUTCOME
mort.drop(['OUTCOME'], axis='columns', inplace=True)

mort = mort.drop('Status', axis=1)
mort = mort.drop('State', axis=1)
mort = mort.drop('orig_apprd_val_amt', axis=1)

mort

c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Bo_Age,Ln_Orig,Orig_LTV_Ratio_Pct,Credit_score,First_home,Tot_mthly_debt_exp,Tot_mthly_incm,pur_prc_amt,DTI Ratio,Median_state_inc,UPB>Appraisal
0,37,75650.0,85,669,0,1707,6000,160000,0.284500,40171,0
1,46,390775.0,102,684,0,0,5025,309000,0.000000,44228,1
2,30,112500.0,90,662,1,1812,4800,176450,0.377500,49894,0
3,24,85250.0,97,647,0,3395,6934,110000,0.489616,43217,0
4,35,114000.0,100,791,0,3801,5504,103000,0.690589,57352,1
...,...,...,...,...,...,...,...,...,...,...,...
10602,37,280200.0,95,652,1,727,1827,80000,0.397920,42590,1
10603,30,57000.0,94,589,0,1703,3941,108000,0.432124,45787,0
10604,33,153200.0,95,675,1,1726,3707,154000,0.465606,43217,0
10605,38,158850.0,100,786,1,1364,2500,92000,0.545600,40171,1


In [18]:
# X, y = mort, outcome
## training data와 test data 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mort, outcome, test_size=0.25, random_state=0)

## Standardization
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [19]:
## 오버샘플링
from imblearn.over_sampling import SMOTE
sm = SMOTE(ratio='auto', kind='regular')
X_resampled, y_resampled = sm.fit_sample(X_train_std, y_train)

In [20]:
pd.Series(y_resampled).value_counts()

1    7446
0    7446
dtype: int64

In [22]:
forest = RandomForestClassifier(criterion='gini', n_estimators=500, max_leaf_nodes=16, random_state=1)
forest.fit(X_resampled, y_resampled)

pred = forest.predict(X_test_std)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95      2502
           1       0.35      0.64      0.45       150

    accuracy                           0.91      2652
   macro avg       0.66      0.78      0.70      2652
weighted avg       0.94      0.91      0.92      2652



In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

array([[2322,  180],
       [  54,   96]], dtype=int64)

In [13]:
forest = RandomForestClassifier(criterion='gini', n_estimators=500, max_leaf_nodes=16, random_state=1)

param_grid = {
    'n_estimators':[200, 500, 800],
    'max_leaf_nodes':[16, 32, 64]
}

results = GridSearchCV(forest, param_grid, cv=5).fit(X_resampled, y_resampled)

In [14]:
## Grid Search 결과를 csv파일로
cv_result = pd.DataFrame(results.cv_results_)
cv_result.to_csv("cv_result_.csv")

PermissionError: [Errno 13] Permission denied: 'cv_result_.csv'

In [15]:
results.best_params_

{'max_leaf_nodes': 64, 'n_estimators': 200}

In [8]:
pred_ = results.predict(X_test_std)
print(classification_report(y_test, pred_))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95      2495
           1       0.38      0.71      0.49       157

    accuracy                           0.91      2652
   macro avg       0.68      0.82      0.72      2652
weighted avg       0.95      0.91      0.93      2652



In [10]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_)

array([[2309,  186],
       [  45,  112]], dtype=int64)